In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
from sklearn import preprocessing

# Setup
num_neuron = 128
learn_rate = 0.001

x_disp = np.loadtxt('data_incompressible/m_rose_nu_05/disp_coord')
y_disp = np.loadtxt('data_incompressible/m_rose_nu_05/disp_data')
x_elas = np.loadtxt('data_incompressible/m_rose_nu_05/strain_coord')
y_elas = np.loadtxt('data_incompressible/m_rose_nu_05/m_data')

ss_x = preprocessing.StandardScaler()
x_disp = ss_x.fit_transform(x_disp.reshape(-1, 2))
x_elas = ss_x.fit_transform(x_elas.reshape(-1, 2))

# Define neural network architectures
class ElasticityNetwork(nn.Module):
    def __init__(self):
        super(ElasticityNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, num_neuron), nn.ReLU(),
            *[nn.Sequential(nn.Linear(num_neuron, num_neuron), nn.ReLU()) for _ in range(11)],
            nn.Linear(num_neuron, 1)
        )

    def forward(self, x):
        return self.layers(x)

class DisplacementNetwork(nn.Module):
    def __init__(self):
        super(DisplacementNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, num_neuron), nn.SiLU(),
            *[nn.Sequential(nn.Linear(num_neuron, num_neuron), nn.SiLU()) for _ in range(11)],
            nn.Linear(num_neuron, 1)
        )

    def forward(self, x):
        return self.layers(x)

elasticity_net = ElasticityNetwork()
disp_net = DisplacementNetwork()

# Define optimizer
optimizer = optim.Adam(list(elasticity_net.parameters()) + list(disp_net.parameters()), lr=learn_rate)

# Read displacements
y_u = torch.tensor(y_disp[:, 0], dtype=torch.float32)
y_disp_tensor = torch.tensor(y_disp, dtype=torch.float32)

# Calculate strains (define conv2d function)
def conv2d(x, weight):
    return nn.functional.conv2d(x, weight, stride=1, padding=0)

# Initialize conv weights (you will need to adapt this part according to your original logic)
conv_x = torch.tensor([[[-0.5]], [[-0.5]]], dtype=torch.float32)
conv_y = torch.tensor([[[0.5]], [[-0.5]]], dtype=torch.float32)

# Training process
start_time = time.time()
for i in range(10): #200001
    optimizer.zero_grad()
    
    # Forward pass
    x_elas_tensor = torch.tensor(x_elas, dtype=torch.float32)
    x_disp_tensor = torch.tensor(x_disp, dtype=torch.float32)
    
    y_pred_m = elasticity_net(x_elas_tensor)
    y_pred_v = disp_net(x_disp_tensor)

    # Calculate strains here (you'll need to implement these calculations)
    # For example, calculating fx_conv_sum_norm and fy_conv_sum_norm:
    # Placeholder calculations; replace these with your actual logic
    fx_conv_sum_norm = torch.randn_like(y_pred_m)  # Example tensor, replace with real calculation
    fy_conv_sum_norm = torch.randn_like(y_pred_v)  # Example tensor, replace with real calculation

    # Calculate loss
    mean_modu = torch.mean(torch.tensor(y_elas, dtype=torch.float32))
    loss_x = torch.mean(torch.abs(fx_conv_sum_norm))
    loss_y = torch.mean(torch.abs(fy_conv_sum_norm))
    loss_m = torch.abs(torch.mean(y_pred_m) - mean_modu)
    loss_v = torch.abs(torch.mean(y_pred_v))
    loss = loss_x + loss_y + loss_m / 100 + loss_v / 100
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    #if i % 100 == 0:
    print(i, loss.item())

# Save predictions
np.savetxt('y_pred_m_final', y_pred_m.detach().numpy())
np.savetxt('y_pred_v_final', y_pred_v.detach().numpy())
print("--- %s Elapsed time ---" % (time.time() - start_time))

0 1.5948041677474976
1 1.5971888303756714
2 1.5990180969238281
3 1.5948235988616943
4 1.6020146608352661
5 1.5955390930175781
6 1.5999568700790405
7 1.5987619161605835
8 1.5939866304397583
9 1.5952712297439575
10 1.5999443531036377
11 1.5940736532211304
12 1.5952750444412231


KeyboardInterrupt: 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
from sklearn import preprocessing

# Setup
num_neuron = 128
learn_rate = 0.0001  # Increased learning rate for faster convergence

x_disp = np.loadtxt('data_incompressible/m_rose_nu_05/disp_coord')
y_disp = np.loadtxt('data_incompressible/m_rose_nu_05/disp_data')
x_elas = np.loadtxt('data_incompressible/m_rose_nu_05/strain_coord')
y_elas = np.loadtxt('data_incompressible/m_rose_nu_05/m_data')

ss_x = preprocessing.StandardScaler()
x_disp = ss_x.fit_transform(x_disp.reshape(-1, 2))
x_elas = ss_x.fit_transform(x_elas.reshape(-1, 2))

# Define neural network architectures with batch normalization
class ElasticityNetwork(nn.Module):
    def __init__(self):
        super(ElasticityNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, num_neuron),
            nn.ReLU(),
            *[nn.Sequential(
                nn.Linear(num_neuron, num_neuron),
                nn.BatchNorm1d(num_neuron),
                nn.ReLU()) for _ in range(6)],  # Reduced number of layers to 6
            nn.Linear(num_neuron, 1)
        )

    def forward(self, x):
        return self.layers(x)

class DisplacementNetwork(nn.Module):
    def __init__(self):
        super(DisplacementNetwork, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(2, num_neuron),
            nn.SiLU(),
            *[nn.Sequential(
                nn.Linear(num_neuron, num_neuron),
                nn.BatchNorm1d(num_neuron),
                nn.SiLU()) for _ in range(6)],  # Reduced number of layers to 6
            nn.Linear(num_neuron, 1)
        )

    def forward(self, x):
        return self.layers(x)

elasticity_net = ElasticityNetwork()
disp_net = DisplacementNetwork()

# Define optimizer and learning rate scheduler
optimizer = optim.Adam(list(elasticity_net.parameters()) + list(disp_net.parameters()), lr=learn_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=100, factor=0.5, verbose=True)

# Read displacements
y_u = torch.tensor(y_disp[:, 0], dtype=torch.float32)
y_disp_tensor = torch.tensor(y_disp, dtype=torch.float32)

# Initialize conv weights
conv_x = torch.tensor([[[-0.5]], [[-0.5]]], dtype=torch.float32)
conv_y = torch.tensor([[[0.5]], [[-0.5]]], dtype=torch.float32)

# Training process
start_time = time.time()
for i in range(10000): #200001
    optimizer.zero_grad()
    
    # Forward pass
    x_elas_tensor = torch.tensor(x_elas, dtype=torch.float32)
    x_disp_tensor = torch.tensor(x_disp, dtype=torch.float32)
    
    y_pred_m = elasticity_net(x_elas_tensor)
    y_pred_v = disp_net(x_disp_tensor)

    # Placeholder calculations for strain (replace with actual logic)
    fx_conv_sum_norm = torch.randn_like(y_pred_m)  # Example tensor, replace with real calculation
    fy_conv_sum_norm = torch.randn_like(y_pred_v)  # Example tensor, replace with real calculation

    # Calculate loss
    mean_modu = torch.mean(torch.tensor(y_elas, dtype=torch.float32))
    loss_x = torch.mean(torch.abs(fx_conv_sum_norm))
    loss_y = torch.mean(torch.abs(fy_conv_sum_norm))
    loss_m = torch.abs(torch.mean(y_pred_m) - mean_modu)
    loss_v = torch.abs(torch.mean(y_pred_v))
    loss = loss_x + loss_y + loss_m / 100 + loss_v / 100
    
    # Backward pass and optimization
    loss.backward()

    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(list(elasticity_net.parameters()) + list(disp_net.parameters()), max_norm=1.0)
    
    optimizer.step()
    
    # Step the learning rate scheduler
    scheduler.step(loss)

    
    if i % 100 == 0:
        print(i, loss.item())

# Save predictions
np.savetxt('y_pred_m_final', y_pred_m.detach().numpy())
np.savetxt('y_pred_v_final', y_pred_v.detach().numpy())
print("--- %s Elapsed time ---" % (time.time() - start_time))


C:\Users\gabri\anaconda3\envs\pytorch_env\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


0 1.6004438400268555
100 1.5872392654418945
200 1.594727635383606
300 1.601154088973999
400 1.5994560718536377
500 1.5987451076507568
600 1.5962367057800293
700 1.5953325033187866
800 1.5978975296020508
900 1.595594882965088
1000 1.5939284563064575
1100 1.5992685556411743
1200 1.5984747409820557
1300 1.593492865562439
1400 1.5930380821228027
1500 1.5954298973083496
1600 1.5909807682037354
1700 1.5973689556121826
1800 1.5965476036071777
1900 1.5970220565795898
2000 1.5918742418289185
2100 1.596195936203003
2200 1.596062421798706
2300 1.5967882871627808
2400 1.6005299091339111
2500 1.5984811782836914
2600 1.5967075824737549
2700 1.5941182374954224
2800 1.597257375717163
2900 1.5938963890075684
3000 1.5935802459716797
3100 1.597844123840332
3200 1.5998713970184326
3300 1.5931522846221924
3400 1.589931845664978
3500 1.5934077501296997
3600 1.5973389148712158
3700 1.5942078828811646
3800 1.5936613082885742
3900 1.6029969453811646
4000 1.5923969745635986
4100 1.590930700302124
4200 1.5950999